In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
punch = pd.read_csv('data/punch.csv', header = None)
flex = pd.read_csv('data/flex.csv', header = None)
print(punch)
print(flex)

         0     1     2      3      4      5
0     1.96  0.62  0.94  69.58  98.39  55.54
1     1.84  0.68  0.87  77.94  30.09  55.24
2     1.53  0.71  0.81  73.18 -22.10  40.71
3     1.21  0.71  0.72  52.25 -39.25  19.35
4     0.98  0.69  0.62  20.39 -35.34  -0.92
...    ...   ...   ...    ...    ...    ...
6995 -0.31  0.96 -0.15   0.79   0.18   2.32
6996 -0.36  0.96 -0.13   2.50  -1.04   1.04
6997 -0.36  0.95 -0.14   2.14   0.00   0.49
6998 -0.37  0.95 -0.15   0.43   0.67   1.77
6999 -0.37  0.95 -0.16  -1.34  -1.34   0.67

[7000 rows x 6 columns]
         0     1     2       3       4       5
0    -0.49  1.55 -1.63 -216.99  367.44 -317.70
1    -0.90 -1.98 -1.87 -204.29  170.96 -265.88
2    -1.01 -1.98 -1.87 -214.42  -27.53 -171.76
3    -1.09 -1.98 -1.87 -206.00 -166.39  -93.57
4    -1.00 -1.98 -1.70 -153.81 -160.71  -39.12
...    ...   ...   ...     ...     ...     ...
6995 -0.77  0.62  0.32    9.28    5.92    1.28
6996 -0.84  0.62  0.31   17.33   -4.27   -3.91
6997 -0.90  0.61  0.29  

In [3]:
SAMPLES_PER_GESTURE = 70

In [4]:
def processData(d, v):
    dataX = np.empty([0,SAMPLES_PER_GESTURE*6])
    dataY = np.empty([0])

    data  = d.values
    dataNum = data.shape[0] // SAMPLES_PER_GESTURE


    for i in tqdm(range(dataNum)):
        tmp = []
        for j in range(SAMPLES_PER_GESTURE):
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][0] + 4.0) / 8.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][1] + 4.0) / 8.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][2] + 4.0) / 8.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][3] + 2000.0) / 4000.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][4] + 2000.0) / 4000.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][5] + 2000.0) / 4000.0]

        tmp = np.array(tmp)

        tmp = np.expand_dims(tmp, axis = 0)

        dataX = np.concatenate((dataX, tmp), axis = 0)
        dataY = np.append(dataY, v)

    return dataX, dataY

In [5]:
punchX, punchY = processData(punch, 0)
flexX, flexY = processData(flex, 1)
dataX = np.concatenate((punchX, flexX), axis = 0)
dataY = np.concatenate((punchY, flexY), axis = 0)

100%|██████████| 100/100 [00:00<00:00, 1708.58it/s]


In [6]:
permutationTrain = np.random.permutation(dataX.shape[0])
print(permutationTrain)

[ 27 122  32 196 140 190 100 193 101 144  73  21 112 159 171  22 180   0
  47 103  93  71  60 172  87  74  72 179  46 156  30 178  25 107  79  26
 149 174  40  41 133  11  77  49  80  69 176  67  43  97 160  20 139  56
 116  48 106 147  44   7  76  55 195  42 175 154 173 123 150  90  99 125
 168 121  92 130  36 135  70 117  37 189 186 141  65 120 151 114  66 188
  34  78 164 109 129  52 110  53 184  84  10  50 167  85 108  63  13 152
 170  81   9 165 124 118 134 183  35 181 148 161  91  17 182  24  16 191
 157 199 158 113  59 119 136  19  96 192   8  39   4  61  45 128 185 104
 162 142  89  12  38  62  18 105 194   6  82  83  23  64 163 145 169  58
  51 177 126 111  31 153  14  75   5 197  28  33  29 132   3 146  57 187
  68  15 198 102 131  95 166  94 137  86   2  98 127 143   1  54 138 155
  88 115]


In [7]:
dataX = dataX[permutationTrain]
dataY = dataY[permutationTrain]
print(dataY)

[0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1.
 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0.
 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 1. 0. 1.]


In [8]:
vfoldSize = int(dataX.shape[0]/100*20)

xTest = dataX[0:vfoldSize]
yTest = dataY[0:vfoldSize]

xTrain = dataX[vfoldSize:dataX.shape[0]]
yTrain = dataY[vfoldSize:dataY.shape[0]]

In [9]:
model = keras.Sequential()
model.add(keras.layers.Dense(32, input_shape =(6*SAMPLES_PER_GESTURE,), activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))
adam = keras.optimizers.Adam()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=adam,
              metrics=['sparse_categorical_accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                13472     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 14,034
Trainable params: 14,034
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(xTrain, yTrain, batch_size=1, validation_data=(xTest, yTest), epochs=200, verbose=1)

Train on 160 samples, validate on 40 samples
Epoch 1/200
160/160 [==============================] - 1s 9ms/sample - loss: 0.7605 - sparse_categorical_accuracy: 0.4812 - val_loss: 0.6861 - val_sparse_categorical_accuracy: 0.5000
Epoch 2/200
160/160 [==============================] - 1s 3ms/sample - loss: 0.7012 - sparse_categorical_accuracy: 0.4812 - val_loss: 0.6833 - val_sparse_categorical_accuracy: 0.9000
Epoch 3/200
160/160 [==============================] - 1s 3ms/sample - loss: 0.6795 - sparse_categorical_accuracy: 0.6375 - val_loss: 0.6829 - val_sparse_categorical_accuracy: 0.5000
Epoch 4/200
160/160 [==============================] - 1s 3ms/sample - loss: 0.6702 - sparse_categorical_accuracy: 0.6000 - val_loss: 0.6597 - val_sparse_categorical_accuracy: 0.5000
Epoch 5/200
160/160 [==============================] - 1s 4ms/sample - loss: 0.6521 - sparse_categorical_accuracy: 0.6625 - val_loss: 0.6408 - val_sparse_categorical_accuracy: 0.5000
Epoch 6/200
160/160 [===================

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

open("model", "wb").write(tflite_model)

57716

In [13]:
!xxd -i model >> model.h